# 

In [4]:
df <- read.table("Faults.NNA", col.names = as.vector(read.table("Faults27x7_var")$V1))
df

X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,⋯,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
42,50,270900,270944,267,17,44,24220,76,108,⋯,0.8182,-0.2913,0.5822,1,0,0,0,0,0,0
645,651,2538079,2538108,108,10,30,11397,84,123,⋯,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
829,835,1553913,1553931,71,8,19,7972,99,125,⋯,0.6667,-0.1228,0.2150,1,0,0,0,0,0,0
853,860,369370,369415,176,13,45,18996,99,126,⋯,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
1289,1306,498078,498335,2409,60,260,246930,37,126,⋯,0.9338,-0.1992,1.0000,1,0,0,0,0,0,0
430,441,100250,100337,630,20,87,62357,64,127,⋯,0.8736,-0.2267,0.9874,1,0,0,0,0,0,0
413,446,138468,138883,9052,230,432,1481991,23,199,⋯,0.9205,0.2791,1.0000,1,0,0,0,0,0,0
190,200,210936,210956,132,11,20,20007,124,172,⋯,0.5000,0.1841,0.3359,1,0,0,0,0,0,0
330,343,429227,429253,264,15,26,29748,53,148,⋯,0.5000,-0.1197,0.5593,1,0,0,0,0,0,0
